# Brain f-MRI Image Classification of Alzheimer's Diagnosis Using Deep Learning Modeling CNNs

## Imports

In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras import datasets, layers, models, regularizers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
from numpy.random import seed
seed(72)
import tensorflow as tf
tf.random.set_seed(72)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Splitting Dataset

In [4]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/drive/MyDrive/data',
    batch_size=6400,
    seed=42,
    image_size=(256, 256)
)

Found 6400 files belonging to 2 classes.


In [5]:
images, labels = next(iter(data))

In [6]:
images, labels = np.array(images), np.array(labels)

In [7]:
images.shape

(6400, 256, 256, 3)

In [8]:
labels.shape

(6400,)

In [9]:
train_images, test_images, train_labels, test_labels = train_test_split(images,
                                                                        labels,
                                                                        test_size=0.3,
                                                                        random_state=42)

In [10]:
test_images, val_images, test_labels, val_labels = train_test_split(test_images,
                                                                    test_labels,
                                                                    random_state=42,
                                                                    test_size=0.5)

In [11]:
input_layer = (256,256,3)

Normalizing images

In [12]:
train_images, test_images, val_images = train_images/255, test_images/255, val_images/255

Functions

Function for evaluating 

In [13]:
def evaluate(model, name, history, X, y, threshold=0.5):
    
    print(f"Results for {name} with threshold = {threshold}.")
    
    plt.rcParams.update({'font.size': 18})
    #Create a function that provides useful vis for model
    #performance. This is especially useful as we are most
    #concerned with the number of false negatives
    
    train_loss=[value for key, value in history.items() if 'loss' in key.lower()][0]
    valid_loss=[value for key, value in history.items() if 'loss' in key.lower()][1]
    train_auc=[value for key, value in history.items() if 'auc' in key.lower()][0]
    valid_auc=[value for key, value in history.items() if 'auc' in key.lower()][1]
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(25, 15))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.plot(train_loss, color='tab:blue', label='Train Loss')
    ax1.plot(valid_loss, color='tab:orange', label='Valid Loss')
    ax1.legend(loc='upper left')
    ax1.set_ylim([0,1.05])
    plt.title('Model Loss')

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('AUC')
    ax2.plot(train_auc, color='tab:blue', label='Train AUC')
    ax2.plot(valid_auc, color='tab:orange', label='Valid AUC')
    ax2.legend(loc='upper left')
    ax2.set_ylim([0.5,1.05])
    plt.title('Model AUC')
        
    y_pred = model.predict(X)
    y_pred_adjusted = np.zeros([len(y), ])
    i=0
    for pred in y_pred:
        if pred > threshold:
            y_pred_adjusted[i] = 1
            i+=1
        else:
            y_pred_adjusted[i] = 0
            i+=1

    cm = confusion_matrix(y, y_pred_adjusted)
    cm_df = pd.DataFrame(cm)

    sns.heatmap(cm, ax=ax3, annot=True, cmap='Blues', fmt='0.7g') 

    plt.sca(ax3)
    plt.xlabel('Predicted Values')
    plt.ylabel('True Values')
    
    train_recall=[value for key, value in history.items() if 'recall' in key.lower()][0]
    valid_recall=[value for key, value in history.items() if 'recall' in key.lower()][1]
    train_precision=[value for key, value in history.items() if 'precision' in key.lower()][0]
    valid_precision=[value for key, value in history.items() if 'precision' in key.lower()][1]
    
    if (cm_df[0][1] == 0) or (cm_df[1][1] ==0):
        train_f1 = 'N/A'
        valid_f1 = 'N/A'
    else:
        train_f1 = 2*(train_recall[-1]*train_precision[-1])/(train_recall[-1]+train_precision[-1])
        valid_f1 = 2*(valid_recall[-1]*valid_precision[-1])/(valid_recall[-1]+valid_precision[-1])
    
    print(f"\n Train f1: {train_f1} \n Val f1: {valid_f1} \n\n Train Recall: {train_recall[-1]} \n Val Recall: {valid_recall[-1]}")

    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Recall')
    ax4.set_ylim([-0.05,1.05])
    ax4.plot(train_recall, '--', color='tab:orange', label='Train Recall')
    ax4.plot(valid_recall, color='tab:orange', label='Valid Recall')
    ax4.legend(loc='lower left')
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

Modeling

First Simple Model

In [ ]:
model1 = models.Sequential([
    # since Conv2D is the first layer of the neural network, we should also specify the size of the input
    layers.Conv2D(128, (3,3), activation='relu', input_shape=(input_layer)),
    # flatten the result to feed it to the dense layer
    layers.Flatten(), 
    # and define 512 neurons for processing the output coming by the previous layers
    layers.Dense(512, activation='relu'),
    # a single output neuron
    layers.Dense(1, activation='sigmoid')  
])

In [ ]:
metrics = ['accuracy', 'AUC', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

In [ ]:
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)

In [ ]:
history1=model1.fit(train_images, 
                  train_labels,
                  epochs=10,
                  validation_data=(val_images,val_labels),
                  verbose=2
                  ).history

In [ ]:
evaluate(model1, 'model1', history1, X=val_images, y=val_labels, threshold=0.5)

Another Model

In [ ]:
model2 = models.Sequential([
    # since Conv2D is the first layer of the neural network, we should also specify the size of the input
    layers.Conv2D(128, (3,3), activation='relu', input_shape=(input_layer)),
    # apply pooling
    layers.MaxPooling2D(2,2),
    # flatten the result to feed it to the dense layer
    layers.Flatten(), 
    # and define 512 neurons for processing the output coming by the previous layers
    layers.Dense(512, activation='relu'),
    # dropout layer
    layers.Dropout(0.5),
    # a single output neuron
    layers.Dense(1, activation='sigmoid')  
])

In [ ]:
model2.summary()

In [ ]:
metrics

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=metrics)

In [ ]:
history2=model2.fit(train_images, 
                  train_labels,
                  epochs=10,
                  validation_data=(val_images,val_labels),
                  verbose=2
                  ).history

In [ ]:
evaluate(model2, 'model2', history2, X=val_images, y=val_labels, threshold=0.5)